<a href="https://colab.research.google.com/github/123shwetarohokale/563-ShwetaR/blob/main/long%20time%20taken%20project%20code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import numpy as np
import matplotlib.pyplot as plt

def pbc(d, L):
    for i, x in enumerate(d):
        if x > L / 2:
            d[i] = x - L
        elif x <= -L / 2:
            d[i] = x + L
    return d

def distance(ri, rj, L):
    d = pbc(rj - ri, L)
    return np.linalg.norm(d)

def lj(r, epsilon=148, sigma=3.73):
    return 4 * epsilon * ((sigma / r) ** 12 - (sigma / r) ** 6)

def autocorr1D(array):
    ft = np.fft.rfft(array - np.average(array))
    acorr = np.fft.irfft(ft * np.conjugate(ft)) / (len(array) * np.var(array))
    dt = np.where(acorr<0)[0][0]
    nsamples = len(array)//dt
    acorr = acorr[dt:nsamples]
    return nsamples, dt

class MethaneSimulation:
    def __init__(self, N=100, L=50, T=300, maxD=0.1, maxSteps=1000, debug=False, seed=2):
        np.random.seed(seed)
        self.N = N
        self.L = L
        self.T = T
        self.maxD = maxD
        self.accept = 0
        self.reject = 0
        self.rcut = 2.5 * 3.73  # Adjusted for methane's sigma
        self.pos = np.zeros((N, 3))
        self.debug = debug
        self.maxSteps = maxSteps
        self.energies = np.zeros(maxSteps)

        for i in range(self.N):
            while True:
                self.pos[i] = np.random.uniform(low=-L/2, high=L/2, size=3)
                if self.energy(i, initializing=True) <= 0.:
                    break

        self.E = self.energy()

        for i in range(self.maxSteps):
            self.trial()
            if self.debug:
                print("{:03}:   {:.2f}".format(i, self.E))
            self.energies[i] = self.E

    def energy(self, i=None, initializing=False):
        E = 0
        if i is not None and initializing:
            pi = self.pos[i]
            for j in range(i):
                r = distance(pi, self.pos[j], self.L)
                if r <= self.rcut:
                    E += lj(r)
            return E

        if i is None:
            for i in range(self.N - 1):
                pi = self.pos[i]
                for j in range(i + 1, self.N):
                    r = distance(pi, self.pos[j], self.L)
                    if r <= self.rcut:
                        E += lj(r)
            return E

    def trial(self):
        i = np.random.randint(self.N)
        oldP = np.copy(self.pos[i])
        oldE = self.E
        oldEi = self.energy()
        d = np.random.uniform(low=-self.maxD, high=self.maxD, size=3)
        newP = pbc(oldP + d, self.L)
        self.pos[i] = newP
        newEi = self.energy()
        dE = newEi - oldEi

        if np.random.uniform() <= np.exp(-dE / self.T):
            if self.debug:
                print("accept")
            self.accept += 1
            self.E += dE
        else:
            if self.debug:
                print("reject")
            self.pos[i] = oldP
            self.reject += 1
            self.E = oldE
# Methane-specific values
density = 0.5
N = 32
V = N / density
L = V ** (1/3)
print(L)

maxSteps = 10000
s = MethaneSimulation(N=N, L=L, maxSteps=maxSteps, debug=False, seed=23)
print("{:02f} +/- {:02f}".format(s.energies.mean() / N, s.energies.std() / N))

3.9999999999999996


KeyboardInterrupt: 